In [1]:
import os

In [2]:
%pwd

'c:\\Users\\H P\\Desktop\\Machine Learning Projects\\ML Ops Project\\Chest_Cancer_Classification_Using_MLflow_-_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\H P\\Desktop\\Machine Learning Projects\\ML Ops Project\\Chest_Cancer_Classification_Using_MLflow_-_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest_Scan_CT")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-07-14 11:33:50,936: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-07-14 11:33:50,977: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-07-14 11:33:50,979: INFO: common: created directory at: artifacts:]
[2025-07-14 11:33:50,982: INFO: common: created directory at: artifacts\training:]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/10
17/17 [==============================] - 74s 4s/step - loss: 18.8814 - accuracy: 0.4749 - val_loss: 16.2167 - val_accuracy: 0.6094
Epoch 2/10
17/17 [==============================] - 80s 5s/step - loss: 15.8317 - accuracy: 0.4865 - val_loss: 0.2558 - val_accuracy: 0.9219
Epoch 3/10
17/17 [==============================] - 73s 4s/step - loss: 6.5204 - accuracy: 0.6834 - val_loss: 23.3070 - val_accuracy: 0.6094
Epoch 4/10
17/17 [==============================] - 74s 4s/step - loss: 6.7725 - accuracy: 0.7220 - val_loss: 6.8950 - val_accuracy: 0.6094
Epoch 5/10
17/17 [===

In [ ]:
## In case if the training data directory is empty or does not contain subdirectories then only the following code will be executed

"""
from pathlib import Path
import shutil

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    print("Training data directory:", training_config.training_data)
    contents = list(Path(training_config.training_data).glob("*"))
    print("Contents of training data directory:", contents)

    # Check if there are any subdirectories (classes) in the training data directory
    class_dirs = [d for d in contents if d.is_dir()]
    if not class_dirs:
        # Automatically create a subdirectory and move all images into it
        unknown_class_dir = Path(training_config.training_data) / "unknown"
        unknown_class_dir.mkdir(exist_ok=True)
        for item in contents:
            if item.is_file():
                shutil.move(str(item), str(unknown_class_dir / item.name))
        print(f"Moved all images into {unknown_class_dir}.")
        # Update contents and class_dirs after moving
        contents = list(Path(training_config.training_data).glob("*"))
        class_dirs = [d for d in contents if d.is_dir()]
        if not class_dirs:
            raise ValueError(f"Failed to create class subdirectory in {training_config.training_data}.")

    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    print("Error:", e)
    raise e
    """